## New prepare & explore using weekend_dataset.csv

In [2]:
# imports

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


import unicodedata
import re

from bs4 import BeautifulSoup
import requests
import os
import json

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

import prepare as prep

from sklearn.model_selection import train_test_split
import sklearn.model_selection

from scipy import stats
from scipy.stats import norm, binom



from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier

In [3]:
# original df

df = pd.read_csv('weekend_dataset.csv', index_col = 0)

In [53]:
# original df
df = pd.read_csv('weekend_dataset.csv')

# combining review cols
df['reviews'] = df['Review 1'] + ' ' + df['Review 2'] + ' ' + df['Review 3'] + ' ' + df['Review 4'] + ' ' +  df['Review 5']

# dropping cols
df.drop(['Review 1', 'Review 2', 'Review 3', 'Review 4', 'Review 5'], axis = 1, inplace = True)

# renaming columns
df = df.rename(columns = {'Book Name': 'title','Synopsis':'summary', 'Link':'link', 'page_count':'length'})

In [20]:
df.head()

,Book Name,Synopsis,page_count,year,review_count,rating,rating_count,genre,author,publisher,Link,scraped_at,reviews
0,Raintree County,"Throughout a single day in 1892, John Shawness...","1088 pages, Paperback","First published January 1, 1948",91,3.95,"1,035 ratings",Fiction,Ross Lockridge Jr.,"April 1, 1994 by Penguin Books",https://www.goodreads.com/book/show/257233.Rai...,2023-03-25 01:31:35.686587,"Did they stop to think, in the midst of their ..."
1,The Eliots of Damerosehay #2\nPilgrim's Inn,"After WW II, Lucilla Eliot's soldier son Georg...","352 pages, Hardcover","First published January 1, 1948",238,4.35,"1,895 ratings",Fiction,Elizabeth Goudge,"March 21, 2013 by Amereon Limited",https://www.goodreads.com/book/show/205621.Pil...,2023-03-25 01:31:52.336480,I think I will start a bookshelf containing bo...
2,"An A.J. Cronin Trilogy: The Northern Light, Th...",Omnibus edition of two unabridged novels: The ...,Hardcover,"Published January 1, 1958",0,4.50,2 ratings,NaN,A.J. Cronin,"January 1, 1958 by Little, Brown and Company",https://www.goodreads.com/book/show/2398522.An...,2023-03-25 01:32:06.053123,no_review no_review no_review no_review no_review
3,The Naked and the Dead by Norman Mailer Lesson...,The Naked and the Dead lesson plan contains a ...,"328 pages, Kindle Edition","First published June 18, 2012",0,0.00,0 ratings,NaN,BookRags,"June 18, 2012",https://www.goodreads.com/book/show/20497230-t...,2023-03-25 01:32:19.143691,no_review no_review no_review no_review no_review
4,The Big Fisherman,"It was a calm, early summer noon in the southe...","459 pages, Hardcover","First published January 1, 1936",130,4.08,"1,795 ratings",Historical Fiction,Lloyd C. Douglas,"January 1, 1954",https://www.goodreads.com/book/show/778667.The...,2023-03-25 01:32:32.567861,There was a thing in the first half of the 20t...


In [8]:
# combining review cols

df['reviews'] = df['Review 1'] + ' ' + df['Review 2'] + ' ' + df['Review 3'] + ' ' + df['Review 4'] + ' ' +  df['Review 5']

In [9]:
df.head()

,Synopsis,page_count,year,review_count,rating,rating_count,genre,author,publisher,Review 1,Review 2,Review 3,Review 4,Review 5,Link,scraped_at,reviews
Book Name,,,,,,,,,,,,,,,,,
Raintree County,"Throughout a single day in 1892, John Shawness...","1088 pages, Paperback","First published January 1, 1948",91,3.95,"1,035 ratings",Fiction,Ross Lockridge Jr.,"April 1, 1994 by Penguin Books","Did they stop to think, in the midst of their ...","""Raintree County"" is an American version of To...",This is a hard book to review: it will definit...,Remembering Mom Who Kept Us in Books\n\nAs for...,It may seem odd to be writing a review of a bo...,https://www.goodreads.com/book/show/257233.Rai...,2023-03-25 01:31:35.686587,"Did they stop to think, in the midst of their ..."
The Eliots of Damerosehay #2\nPilgrim's Inn,"After WW II, Lucilla Eliot's soldier son Georg...","352 pages, Hardcover","First published January 1, 1948",238,4.35,"1,895 ratings",Fiction,Elizabeth Goudge,"March 21, 2013 by Amereon Limited",I think I will start a bookshelf containing bo...,The year is 1947 or thereabouts. Britain has w...,"”Increasingly, as she lived here, Nadine had a...",I usually read this book at Christmas. I love ...,Extraordinary fiction. If you haven't read Gou...,https://www.goodreads.com/book/show/205621.Pil...,2023-03-25 01:31:52.336480,I think I will start a bookshelf containing bo...
"An A.J. Cronin Trilogy: The Northern Light, The Green Years, Shannon's Way",Omnibus edition of two unabridged novels: The ...,Hardcover,"Published January 1, 1958",0,4.50,2 ratings,NaN,A.J. Cronin,"January 1, 1958 by Little, Brown and Company",no_review,no_review,no_review,no_review,no_review,https://www.goodreads.com/book/show/2398522.An...,2023-03-25 01:32:06.053123,no_review no_review no_review no_review no_review
The Naked and the Dead by Norman Mailer Lesson Plans,The Naked and the Dead lesson plan contains a ...,"328 pages, Kindle Edition","First published June 18, 2012",0,0.00,0 ratings,NaN,BookRags,"June 18, 2012",no_review,no_review,no_review,no_review,no_review,https://www.goodreads.com/book/show/20497230-t...,2023-03-25 01:32:19.143691,no_review no_review no_review no_review no_review
The Big Fisherman,"It was a calm, early summer noon in the southe...","459 pages, Hardcover","First published January 1, 1936",130,4.08,"1,795 ratings",Historical Fiction,Lloyd C. Douglas,"January 1, 1954",There was a thing in the first half of the 20t...,There is no doubt that Jesus Christ worked ver...,"Read it long ago. In fact, when I became a mem...",This companion and pseudo-prequel to Douglas’ ...,Lloyd C Douglas is quickly becoming one of my ...,https://www.goodreads.com/book/show/778667.The...,2023-03-25 01:32:32.567861,There was a thing in the first half of the 20t...


In [ ]:
# dropping

df.drop(['Review 1', 'Review 2', 'Review 3', 'Review 4', 'Review 5'], axis = 1, inplace = True)

In [33]:
df = df.rename(columns = {'Book Name': 'title','Synopsis':'summary', 'Link':'link', 'page_count':'length'})

In [57]:
df.head()

,Book Name,Synopsis,page_count,year,review_count,rating,rating_count,genre,author,publisher,Link,scraped_at,reviews
0,Raintree County,"Throughout a single day in 1892, John Shawness...","1088 pages, Paperback","First published January 1, 1948",91,3.95,"1,035 ratings",Fiction,Ross Lockridge Jr.,"April 1, 1994 by Penguin Books",https://www.goodreads.com/book/show/257233.Rai...,2023-03-25 01:31:35.686587,"Did they stop to think, in the midst of their ..."
1,The Eliots of Damerosehay #2\nPilgrim's Inn,"After WW II, Lucilla Eliot's soldier son Georg...","352 pages, Hardcover","First published January 1, 1948",238,4.35,"1,895 ratings",Fiction,Elizabeth Goudge,"March 21, 2013 by Amereon Limited",https://www.goodreads.com/book/show/205621.Pil...,2023-03-25 01:31:52.336480,I think I will start a bookshelf containing bo...
2,"An A.J. Cronin Trilogy: The Northern Light, Th...",Omnibus edition of two unabridged novels: The ...,Hardcover,"Published January 1, 1958",0,4.50,2 ratings,NaN,A.J. Cronin,"January 1, 1958 by Little, Brown and Company",https://www.goodreads.com/book/show/2398522.An...,2023-03-25 01:32:06.053123,no_review no_review no_review no_review no_review
3,The Naked and the Dead by Norman Mailer Lesson...,The Naked and the Dead lesson plan contains a ...,"328 pages, Kindle Edition","First published June 18, 2012",0,0.00,0 ratings,NaN,BookRags,"June 18, 2012",https://www.goodreads.com/book/show/20497230-t...,2023-03-25 01:32:19.143691,no_review no_review no_review no_review no_review
4,The Big Fisherman,"It was a calm, early summer noon in the southe...","459 pages, Hardcover","First published January 1, 1936",130,4.08,"1,795 ratings",Historical Fiction,Lloyd C. Douglas,"January 1, 1954",https://www.goodreads.com/book/show/778667.The...,2023-03-25 01:32:32.567861,There was a thing in the first half of the 20t...


In [58]:
df.shape

(4891, 13)

In [59]:
df.isna().sum()

Book Name         0
Synopsis        121
page_count       18
year             32
review_count      0
rating            0
rating_count      0
genre           297
author            0
publisher        25
Link              0
scraped_at        0
reviews         398
dtype: int64